In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import fugashi

text = """
みなさんこんにちは。天地創造はこの巻で終わりとなります。
最後が急転回になってしまって申し訳ありません。
ロイドファンの方にいたってはまことに頭が上がりません。ゲームではもっと男らしくてかっこいいんですが…連載中手紙などではげまして下さった方々本当にありがとうございました。手紙が何よりパワーのミ原になりました。
なかなかお返事は出せませんがが心から感謝いたしております。

とにかくファンタジーを愛しているのでいつかオリジナルでみなさんをアナザーワールドへご招待と夢みています。
また違う世界でお会いでまた時は応援にくださいね。
今年もまんが道で頑張ります！
""".strip()

In [6]:
tagger = fugashi.Tagger()

In [7]:
print(tagger(text))

[みな, さん, こんにちは, 。, 天地, 創造, は, この, 巻, で, 終わり, と, なり, ます, 。, 最後, が, 急, 転回, に, なっ, て, しまっ, て, 申し訳, あり, ませ, ん, 。, ロイド, ファン, の, 方, に, いたっ, て, は, まこと, に, 頭, が, 上がり, ませ, ん, 。, ゲーム, で, は, もっと, 男, らしく, て, かっこ, いい, ん, です, が, …, 連載, 中, 手紙, など, で, はげまし, て, 下さっ, た, 方々, 本当, に, ありがとう, ござい, まし, た, 。, 手紙, が, 何, より, パワー, の, ミ, 原, に, なり, まし, た, 。, なかなか, お, 返事, は, 出せ, ませ, ん, が, が, 心, から, 感謝, いたし, て, おり, ます, 。, とにかく, ファンタジー, を, 愛し, て, いる, の, で, いつ, か, オリジナル, で, みな, さん, を, アナザー, ワールド, へ, ご, 招待, と, 夢み, て, い, ます, 。, また, 違う, 世界, で, お, 会い, で, また, 時, は, 応援, に, ください, ね, 。, 今年, も, まんが, 道, で, 頑張り, ます, ！]


In [8]:
tokens = tagger(text)
tokens[0]

みな

In [9]:
tab = "\t"
for token in tokens:
    print(f"{token}\t{str.join(tab, token.feature_raw.split(','))}")

みな	名詞	普通名詞	副詞可能	*	*	*	ミナ	皆	みな	ミナ	みな	ミナ	和	*	*	*	*	*	*	体	ミナ	ミナ	ミナ	ミナ	"2	0"	C3	*	10081980494520832	36678
さん	接尾辞	名詞的	一般	*	*	*	サン	さん	さん	サン	さん	サン	和	*	*	*	*	*	*	接尾体	サン	サン	サン	サン	*	C4	*	3984363884782080	14495
こんにちは	感動詞	一般	*	*	*	*	コンニチハ	今日は	こんにちは	コンニチワ	こんにちは	コンニチワ	混	*	*	*	*	*	*	他	コンニチハ	コンニチハ	コンニチハ	コンニチハ	5	*	*	37963946107478528	138112
。	補助記号	句点	*	*	*	*	*	。	。	*	。	*	記号	*	*	*	*	*	*	補助	*	*	*	*	*	*	*	6880571302400	25
天地	名詞	普通名詞	一般	*	*	*	テンチ	天地	天地	テンチ	天地	テンチ	漢	*	*	*	*	*	*	体	テンチ	テンチ	テンチ	テンチ	1	C1	*	6992077942563328	25437
創造	名詞	普通名詞	サ変可能	*	*	*	ソウゾウ	創造	創造	ソーゾー	創造	ソーゾー	漢	*	*	*	*	*	*	体	ソウゾウ	ソウゾウ	ソウゾウ	ソウゾウ	0	C2	*	5799107826426368	21097
は	助詞	係助詞	*	*	*	*	ハ	は	は	ワ	は	ワ	和	*	*	*	*	*	*	係助	ハ	ハ	ハ	ハ	*	"動詞%F2@0	名詞%F1	形容詞%F2@-1"	*	8059703733133824	29321
この	連体詞	*	*	*	*	*	コノ	此の	この	コノ	この	コノ	和	*	*	*	*	*	*	相	コノ	コノ	コノ	コノ	0	*	*	3547308012741120	12905
巻	名詞	普通名詞	助数詞可能	*	*	*	マキ	巻	巻	マキ	巻	マキ	和	*	*	*	*	*	*	体	マキ	マキ	マキ	マキ	1	C1	*	9763122122465792	35518
で	助詞	格助詞	*	*	*	*	デ	で	で	デ	で	デ	和	*	*	*	*	*	*	格助	デ	デ	デ	デ	*	"動詞%F2@0	名詞%F1"	*	701434305302

In [10]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 50
token.feature._asdict()

{'pos1': '補助記号',
 'pos2': '句点',
 'pos3': '*',
 'pos4': '*',
 'cType': '*',
 'cForm': '*',
 'lForm': '*',
 'lemma': '！',
 'orth': '！',
 'pron': '*',
 'orthBase': '！',
 'pronBase': '*',
 'goshu': '記号',
 'iType': '*',
 'iForm': '*',
 'fType': '*',
 'fForm': '*',
 'iConType': '*',
 'fConType': '*',
 'type': '補助',
 'kana': '*',
 'kanaBase': '*',
 'form': '*',
 'formBase': '*',
 'aType': '*',
 'aConType': '*',
 'aModType': '*',
 'lid': '12378129441280',
 'lemma_id': '45'}

In [21]:
data = pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T 
                  for token in tagger("先生が言ったのを覚えている？")]).reset_index(drop=True)[["orth", "lemma", "pos1", "pos2", "cType", "kana"]]
data

,orth,lemma,pos1,pos2,cType,kana
0,先生,先生,名詞,普通名詞,*,センセイ
1,が,が,助詞,格助詞,*,ガ
2,言っ,言う,動詞,一般,五段-ワア行,イッ
3,た,た,助動詞,*,助動詞-タ,タ
4,の,の,助詞,準体助詞,*,ノ
5,を,を,助詞,格助詞,*,ヲ
6,覚え,覚える,動詞,一般,下一段-ア行,オボエ
7,て,て,助詞,接続助詞,*,テ
8,いる,居る,動詞,非自立可能,上一段-ア行,イル
9,？,？,補助記号,句点,*,*


In [12]:
attr = ["orth", "lemma", "pos1", "cType", "kana"]

In [13]:
data[attr]

,orth,lemma,pos1,cType,kana
0,みな,皆,名詞,*,ミナ
1,さん,さん,接尾辞,*,サン
2,こんにちは,今日は,感動詞,*,コンニチハ
3,。,。,補助記号,*,*
4,天地,天地,名詞,*,テンチ
5,創造,創造,名詞,*,ソウゾウ
6,は,は,助詞,*,ハ
7,この,此の,連体詞,*,コノ
8,巻,巻,名詞,*,マキ
9,で,で,助詞,*,デ


In [14]:
from pykakasi import kakasi

In [15]:
kks = kakasi()
kks.setMode("K", "H")
conv = kks.getConverter()
conv.do("カタカナ")

'かたかな'

In [16]:
def furiganize(text):
    kks = kakasi()
    kks.setMode("K", "H")
    conv = kks.getConverter()
    
    tagger = fugashi.Tagger()
    
    def convert(toktext):
        from extra import stop_words
        
        token, text = toktext
        feature = token.feature
        
        if feature.orthBase is None:
            out = str(token)
        elif (feature.orthBase, conv.do(feature.kanaBase)) in stop_words:
            out = feature.orth        
        elif feature.kana == "*" or feature.kana == feature.orth:
            out = feature.orth
        else:
            kana = conv.do(feature.kana)
            
            if kana == feature.orth:
                out = feature.orth
            else:
                k = -1
                
                while feature.orth[k] == kana[k]:
                    k -= 1
                    
                k += 1
                
                if k == 0:
                    out = rf"\f{{{feature.orth}}}{{{kana}}}"
                else:                   
                    out = rf"\f{{{feature.orth[:k]}}}{{{kana[:k]}}}{feature.orth[k:]}"
        
        return token, out
            
    def box(toktext):
        token, text = toktext
        return token, rf"\p{{{text}}}"
    
    def wrap(token):
        return token, ""
    
    def finish(toktext):
        token, text = toktext
        return text
    
    toktexts = [convert(wrap(token)) for token in tagger(text)]
    
    prev_token, prev_text = None, None
    box = False
    last = len(toktexts) - 1
    
    def should_close_box(token):
        return (token.feature.pos1 in ["名詞", "代名詞", "形状詞", "連体詞", "副詞", "接続詞", "感動詞", "動詞", "形容詞", "接頭辞"]
                or should_keep_box(token))
    
    def should_keep_box(token):
        return token.feature.pos1 == "補助記号" and token.feature.orth in ["「", "（", "＜", "(", "["]
    
    keep_box = False
    
    for k, (token, text) in enumerate(toktexts):
        if k == 0:
            text = r"\p{" + text
        elif should_close_box(token) and not keep_box:
            text = r"}\p{" + text
            
        keep_box = should_keep_box(token)
            
        if k == last:
            text = text + r"}"
            
        toktexts[k] = token, text
        prev_token, prev_text = token, text
        
    return str.join("", [finish(toktext) for toktext in toktexts])

In [26]:
print(*[furiganize(t) for t in text.split("\n")], sep="\n")

\p{みなさん}\p{こんにちは。}\p{\f{天地}{てんち}}\p{\f{創造}{そうぞう}は}\p{この}\p{\f{巻}{まき}で}\p{\f{終}{お}わりと}\p{なります。}
\p{\f{最後}{さいご}が}\p{\f{急}{きゅう}}\p{\f{転回}{てんかい}に}\p{なって}\p{しまって}\p{\f{申し訳}{もうしわけ}}\p{ありません。}
\p{ロイド}\p{ファンの}\p{\f{方}{ほう}に}\p{いたっては}\p{まことに}\p{\f{頭}{あたま}が}\p{\f{上}{あ}がりません。}\p{ゲームでは}\p{もっと}\p{\f{男}{おとこ}らしくて}\p{かっこ}\p{いいんですが…}\p{\f{連載}{れんさい}\f{中}{ちゅう}}\p{\f{手紙}{てがみ}などで}\p{はげまして}\p{\f{下}{くだ}さった}\p{\f{方々}{かたがた}}\p{\f{本当}{ほんと}に}\p{ありがとう}\p{ございました。}\p{\f{手紙}{てがみ}が}\p{何より}\p{パワーの}\p{ミ}\p{\f{原}{みなもと}に}\p{なりました。}
\p{なかなか}\p{お}\p{\f{返事}{へんじ}は}\p{\f{出}{だ}せませんがが}\p{\f{心}{こころ}から}\p{\f{感謝}{かんしゃ}}\p{いたして}\p{おります。}

\p{とにかく}\p{ファンタジーを}\p{\f{愛}{あい}して}\p{いるので}\p{いつか}\p{オリジナルで}\p{みなさんを}\p{アナザー}\p{ワールドへ}\p{ご}\p{\f{招待}{しょうたい}と}\p{\f{夢}{ゆめ}みて}\p{います。}
\p{また}\p{\f{違}{ちが}う}\p{\f{世界}{せかい}で}\p{お}\p{\f{会}{あ}いで}\p{また}\p{時は}\p{\f{応援}{おうえん}に}\p{くださいね。}
\p{\f{今年}{ことし}も}\p{まんが\f{道}{どう}で}\p{\f{頑張}{がんば}ります！}


In [35]:
text2 = r"""
どうして記事は急いで家に帰ることをお勧めしないのですか？
どこで情報を集めることができますか？
どの様に家族や友達と連絡をとることができますか？
どうして避難所にいる時健康に気を付けなければなりませんか？
エコノミークラス症候群はどういう病気なんですか？
""".strip()

text3= r"""
_____うちまで武君を連れてくれ。
ファイアーウォールがインターネットの_____を保障する。
花子ちゃんは警察に____します。
鈴木先生に何の____？
酷い目にあいたくないなら、武君と____に関わらない方がいいです。
貴重な花瓶が_____武君は困っている。
親と_______住むのがとても不便です。
""".strip()

In [37]:
print(*[r"\item " + furiganize(row) for row in text2.split("\n")], sep="\n")

\item \p{どう}\p{して}\p{\f{記事}{きじ}は}\p{\f{急}{いそ}いで}\p{\f{家}{いえ}に}\p{\f{帰}{かえ}る}\p{ことを}\p{お}\p{\f{勧}{すす}め}\p{しないのですか？}
\item \p{どこで}\p{\f{情報}{じょうほう}を}\p{\f{集}{あつ}める}\p{ことが}\p{できますか？}
\item \p{どの}\p{\f{様}{よう}に}\p{\f{家族}{かぞく}や}\p{\f{友達}{ともだち}と}\p{\f{連絡}{れんらく}を}\p{とる}\p{ことが}\p{できますか？}
\item \p{どう}\p{して}\p{\f{避難}{ひなん}\f{所}{しょ}に}\p{いる}\p{時}\p{\f{健康}{けんこう}に}\p{\f{気}{き}を}\p{\f{付}{つ}けなければ}\p{なりませんか？}
\item \p{エコノミー}\p{クラス}\p{\f{症候}{しょうこう}}\p{\f{群}{ぐん}は}\p{どう}\p{いう}\p{\f{病気}{びょうき}なんですか？}


In [16]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger(text3)]).reset_index(drop=True)[attr]

,orth,lemma,pos1,cType,kana
0,危険,危険,名詞,*,キケン
1,な,だ,助動詞,助動詞-ダ,ナ
2,もの,物,名詞,*,モノ
3,が,が,助詞,*,ガ
4,起こる,起こる,動詞,五段-ラ行,オコル
5,と,と,助詞,*,ト
6,、,、,補助記号,*,*
7,すぐ,直ぐ,副詞,*,スグ
8,に,に,助詞,*,ニ
9,助け,助け,名詞,*,タスケ


In [17]:
from extra import stop_words
stop_words

[('車', 'くるま'),
 ('上', 'うえ'),
 ('下', 'した'),
 ('入る', 'はいる'),
 ('火', 'ひ'),
 ('外', 'そと'),
 ('出る', 'でる'),
 ('出す', 'だす'),
 ('大きい', 'おおきい'),
 ('大きな', 'おおきな'),
 ('山', 'やま'),
 ('中', 'なか'),
 ('何', 'なに'),
 ('時', 'とき'),
 ('見る', 'みる'),
 ('今', 'いま'),
 ('前', 'まえ'),
 ('話す', 'はなす'),
 ('一', 'いち'),
 ('二', 'に'),
 ('三', 'さん'),
 ('四', 'よん'),
 ('五', 'ご'),
 ('六', 'ろく'),
 ('七', 'しち'),
 ('八', 'はち'),
 ('九', 'きゅう'),
 ('十', 'じゅう'),
 ('百', 'ひゃく'),
 ('千', 'せん'),
 ('万', 'まん')]

In [18]:
tagger(text)[0].feature.pos1

'名詞'

In [19]:
import jamdict
jmd = jamdict.Jamdict()

In [20]:
result = jmd.lookup("や")

In [21]:
result.entries[5].senses[0].pos[0]

'particle'

In [22]:
tokens[8].feature.pos1

'助詞'

In [23]:
[entry for entry in result.entries if any(["particle" in sense.pos for sense in entry.senses])]

[や : 1. such things as (non-exhaustive list related to a specific time and place)/and ... and 2. the minute (that) .../no sooner than .../as soon as 3. punctuational exclamation in haiku, renga, etc. 4. interjection expressing surprise 5. hi 6. plain copula 7. yes/what? 8. indicates a question]

In [24]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger("健二郎さんは今夜お酒ばかりを飲んでいます。")]).reset_index(drop=True)[["orth", "lemma", "pos1", "pos2"]]

,orth,lemma,pos1,pos2
0,健二郎,ケンジロウ,名詞,固有名詞
1,さん,さん,接尾辞,名詞的
2,は,は,助詞,係助詞
3,今夜,今夜,名詞,普通名詞
4,お,御,接頭辞,*
5,酒,酒,名詞,普通名詞
6,ばかり,ばかり,助詞,副助詞
7,を,を,助詞,格助詞
8,飲ん,飲む,動詞,一般
9,で,て,助詞,接続助詞


In [35]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger("誰かがいる。")]).reset_index(drop=True)[["orth", "lemma", "pos1", "pos2"]]

,orth,lemma,pos1,pos2
0,誰,誰,代名詞,*
1,か,か,助詞,副助詞
2,が,が,助詞,格助詞
3,いる,居る,動詞,非自立可能
4,。,。,補助記号,句点
